In [1]:
import tensorflow as tf
from tensorflow import keras
from json import loads
import pandas as pd
from keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import numpy as np
from ast import literal_eval
import pickle

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [3]:
with open('../Data/LibriSpeech/my_dictRNN.pkl', 'rb') as f:
    my_dict = pickle.load(f)
my_dict
x=[]
y=[]
for i in my_dict:
    x.append(i['speaker'])
    y.append(i['cepstrum'])
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# create an example list of labels
labels = x
# create a LabelEncoder object
le = LabelEncoder()
# fit the encoder to the labels and transform the labels
x = le.fit_transform(labels)
print(len(le.classes_))
# print the original labels and the encoded labels
print("Original labels:", labels)
print("Encoded labels:", x)
with open('label_encoder.pkl','wb') as f:
    pickle.dump(le, f) 
# Save the encoded data as a NumPy array
np.save('encoded_data.npy', x)

y=np.array(y)
y.shape

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=x, random_state=42)

125
Original labels: [' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen Savage', ' Karen S

In [4]:
timestep=y.shape[1]
feature=y.shape[2]
model=models.Sequential()
model.add(keras.layers.LSTM(units=50,return_sequences=False,input_shape=(timestep,feature)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(units=len(le.classes_)))
model.add(keras.layers.Activation('softmax'))
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                18200     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 125)               6375      
                                                                 
 activation (Activation)     (None, 125)               0         
                                                                 
Total params: 24,575
Trainable params: 24,575
Non-trainable params: 0
_________________________________________________________________


In [5]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy( from_logits=False),
              metrics=['accuracy'])
history=model.fit(y_train,x_train,batch_size=4,epochs=100,validation_data=(y_test,x_test))

Epoch 1/100
2854/2854 [==============================] - 68s 23ms/step - loss: 3.9616 - accuracy: 0.1250 - val_loss: 2.9324 - val_accuracy: 0.3697
Epoch 2/100
2854/2854 [==============================] - 65s 23ms/step - loss: 2.5219 - accuracy: 0.3933 - val_loss: 2.0196 - val_accuracy: 0.5406
Epoch 3/100
2854/2854 [==============================] - 65s 23ms/step - loss: 1.8635 - accuracy: 0.5272 - val_loss: 1.4925 - val_accuracy: 0.6440
Epoch 4/100
2854/2854 [==============================] - 65s 23ms/step - loss: 1.5334 - accuracy: 0.5959 - val_loss: 1.2522 - val_accuracy: 0.6983
Epoch 5/100
2854/2854 [==============================] - 65s 23ms/step - loss: 1.3619 - accuracy: 0.6323 - val_loss: 1.1425 - val_accuracy: 0.6934
Epoch 6/100
2854/2854 [==============================] - 65s 23ms/step - loss: 1.2245 - accuracy: 0.6617 - val_loss: 1.0368 - val_accuracy: 0.7225
Epoch 7/100
2854/2854 [==============================] - 65s 23ms/step - loss: 1.1671 - accuracy: 0.6705 - val_loss: 0

In [6]:
model.save('model.h5')

In [10]:
import soundfile as sf
import numpy as np
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as idp
def MFCC(TRAIN_PATH,file):
    audio, sample_rate, = sf.read(TRAIN_PATH + file)
    def normalize_audio(audio):
        audio = audio / np.max(np.abs(audio))
        return audio
    audio = normalize_audio(audio)
    def frame_audio(audio, FFT_size=2048, hop_size=10, sample_rate=44100):
        # hop_size in ms
        
        audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
        frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
        frame_num = int((len(audio) - FFT_size) / frame_len) + 1
        frames = np.zeros((frame_num,FFT_size))
        
        for n in range(frame_num):
            frames[n] = audio[n*frame_len:n*frame_len+FFT_size]
        
        return frames
    hop_size = 15 #ms
    FFT_size = 2048
    audio_framed = frame_audio(audio, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate)
    window = get_window("hann", FFT_size, fftbins=True)
    audio_win = audio_framed * window
    audio_winT = np.transpose(audio_win)
    audio_fft = np.empty((int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

    for n in range(audio_fft.shape[1]):
        audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

    audio_fft = np.transpose(audio_fft)
    audio_power = np.square(np.abs(audio_fft))
    freq_min = 0
    freq_high = sample_rate / 2
    mel_filter_num = 10
    def freq_to_mel(freq):
        return 2595.0 * np.log10(1.0 + freq / 700.0)

    def met_to_freq(mels):
        return 700.0 * (10.0**(mels / 2595.0) - 1.0)
    def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100):
        fmin_mel = freq_to_mel(fmin)
        fmax_mel = freq_to_mel(fmax)
        
        mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
        freqs = met_to_freq(mels)
        
        return np.floor((FFT_size + 1) / sample_rate * freqs).astype(int), freqs
    filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=44100)
    def get_filters(filter_points, FFT_size):
        filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
        
        for n in range(len(filter_points)-2):
            filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
            filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
        
        return filters
    filters = get_filters(filter_points, FFT_size)
    enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
    filters *= enorm[:, np.newaxis]
    audio_filtered = np.dot(filters, np.transpose(audio_power))
    audio_log = 10.0 * np.log10(audio_filtered)
    def dct(dct_filter_num, filter_len):
        basis = np.empty((dct_filter_num,filter_len))
        basis[0, :] = 1.0 / np.sqrt(filter_len)
        
        samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

        for i in range(1, dct_filter_num):
            basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
            
        return basis
    dct_filter_num = 40

    dct_filters = dct(dct_filter_num, mel_filter_num)
    audio_log=np.nan_to_num(audio_log,posinf=0,neginf=0)

    cepstral_coefficents = np.dot(dct_filters, audio_log)
    return cepstral_coefficents
Test_speaker=MFCC('C:/Users/Admin/Documents/GitHub/Sound-processing/Data/LibriSpeech/testWebData/','26-495-0003.flac')
indices = np.arange(Test_speaker.shape[1])
A=np.transpose(Test_speaker[:,np.random.choice(indices, size=50, replace=False)])
# print(A.shape)
# print(y_test[:1].shape)
# print(y_test[:1])
# print(A)
A=np.expand_dims(A, axis=0)
prediction = model.predict(A)
print("prediction shape:", prediction)
a=np.argmax(prediction)
pred=np.array([a])
print(np.max(prediction))
decoded_value = le.inverse_transform(pred)
print(decoded_value)
prediction

1/1 [==============================] - 0s 54ms/step
prediction shape: [[2.72576813e-21 3.83814136e-09 1.04419877e-08 5.62513858e-09
  1.05150666e-10 2.08438692e-12 1.03095165e-15 8.72302924e-12
  2.15354152e-02 2.78841620e-11 3.44060936e-05 1.01878468e-05
  1.51677471e-09 6.87358642e-11 7.62635409e-07 3.50914699e-17
  2.09937255e-07 3.28750525e-12 4.89849050e-08 3.46840547e-12
  1.03969784e-11 9.03933085e-07 2.00121929e-12 3.12540078e-06
  4.66207385e-11 6.53056134e-07 1.22707945e-07 4.88212146e-03
  3.05165315e-08 7.72927325e-18 5.64690529e-17 5.09675102e-09
  9.58986163e-01 2.57108482e-14 3.42673947e-21 3.50047617e-06
  3.48824407e-11 3.09834247e-09 9.06793929e-08 1.78362779e-06
  1.35492044e-07 3.02192760e-08 6.65724281e-11 1.14635271e-13
  9.04909869e-10 1.74470202e-07 5.09049776e-15 3.03437986e-08
  1.12636294e-02 7.16742814e-14 1.38736056e-15 5.34885811e-22
  1.37470101e-06 1.15658375e-04 1.99887751e-08 3.31588645e-09
  2.90957139e-26 6.95546716e-16 1.27332942e-12 2.19525331e-12


array([[2.72576813e-21, 3.83814136e-09, 1.04419877e-08, 5.62513858e-09,
        1.05150666e-10, 2.08438692e-12, 1.03095165e-15, 8.72302924e-12,
        2.15354152e-02, 2.78841620e-11, 3.44060936e-05, 1.01878468e-05,
        1.51677471e-09, 6.87358642e-11, 7.62635409e-07, 3.50914699e-17,
        2.09937255e-07, 3.28750525e-12, 4.89849050e-08, 3.46840547e-12,
        1.03969784e-11, 9.03933085e-07, 2.00121929e-12, 3.12540078e-06,
        4.66207385e-11, 6.53056134e-07, 1.22707945e-07, 4.88212146e-03,
        3.05165315e-08, 7.72927325e-18, 5.64690529e-17, 5.09675102e-09,
        9.58986163e-01, 2.57108482e-14, 3.42673947e-21, 3.50047617e-06,
        3.48824407e-11, 3.09834247e-09, 9.06793929e-08, 1.78362779e-06,
        1.35492044e-07, 3.02192760e-08, 6.65724281e-11, 1.14635271e-13,
        9.04909869e-10, 1.74470202e-07, 5.09049776e-15, 3.03437986e-08,
        1.12636294e-02, 7.16742814e-14, 1.38736056e-15, 5.34885811e-22,
        1.37470101e-06, 1.15658375e-04, 1.99887751e-08, 3.315886